In [99]:
import pandas as pd
import glob, os, json
import numpy as np
from get_name_and_tag_by_ppuid import get_name_and_tag_by_ppuid

In [72]:
json_match_data_dir = 'data/lor-match-data'
json_pattern = os.path.join(json_match_data_dir, '*.json')
file_list = glob.glob(json_pattern)

In [79]:
dfs = []
for filename in file_list:
    data = pd.read_json(filename)
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)
df_extendida = pd.concat([pd.json_normalize(df.iloc[:,0]),pd.json_normalize(df.iloc[:,1])], axis=1, join="inner")

In [82]:
list_ppuids =  list(np.concatenate(df_extendida.loc[(df_extendida.game_type == 'Ranked') | (df_extendida.game_mode == "SeasonalTournamentLobby")]['participants'].to_list()).flat)

In [86]:
def delete_duplicates_list(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

list_ppuids_no_duplicates = delete_duplicates_list(list_ppuids)

In [95]:
#check list de ppuids y separar por master y diamond+
#primero cargamos los archivos que contienen nuestra info anterior

def readData(filename):
    data = None
    with open(filename, 'r') as fp:
        data = json.load(fp)

    return data

list_master_player_ppuid = readData("data/lor-player-data/master_player_list_ppuids.json")
list_highDiamond_player_ppuid = readData("data/lor-player-data/high_diamond_player_ppuid_list.json")

#Cargamos base de datos del master leaderboard
df_master_leaderboard = pd.json_normalize(pd.read_json('data/master-leaderboard-data/data_americas.json').iloc[:,0])

In [102]:
list_ppuids_no_duplicates[40] not in (list_master_player_ppuid or list_highDiamond_player_ppuid)
ppuid_info = get_name_and_tag_by_ppuid('americas', list_ppuids_no_duplicates[40])

In [104]:
(df_master_leaderboard.iloc[:,0] == ppuid_info['gameName']).any()

False

In [105]:
for player in list_ppuids_no_duplicates:
    if player not in (list_master_player_ppuid or list_highDiamond_player_ppuid):
        try:
            ppuid_info = get_name_and_tag_by_ppuid('americas', player)
            if (df_master_leaderboard.iloc[:,0] == ppuid_info['gameName']).any() == True:
                list_master_player_ppuid.append(player)
            else:
                list_highDiamond_player_ppuid.append(player)
        except:
            print('algun error')

In [112]:
def writeData(filename, data):
    with open(filename, 'w') as file:
         file.write(format(json.dumps(data, ensure_ascii=False, indent=4)))

In [114]:
writeData('data/lor-player-data/master_player_list_ppuids.json', list_master_player_ppuid)
writeData('data/lor-player-data/high_diamond_player_ppuid_list.json', delete_duplicates_list(list_highDiamond_player_ppuid))